<h2>Преобразование переменных</h2>

In [54]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [55]:
df = pd.read_csv('data/final.csv').drop(columns=['Unnamed: 0'])

In [56]:
# не писать в презе
df['finish_dttm'] = pd.to_datetime(df['finish_dttm'], format='%Y-%m-%d %H:%M:%S') # преобразование во временную переменную
df['age'] = pd.to_numeric(df['age'], errors='coerce') # преобразование во числовую переменную
df["bundle_nm"] = df["bundle_nm"].fillna('Без подписки')
df["bundle_nm"] = df["bundle_nm"].replace('XXX', 'Без подписки')

df['success_flg'] = df['wo_hit_status_result_desc'].map({'Дозвон, Отказ': 0, 'Дозвон, Успешно': 1})

# хз писать в презе
df['gender_cd'] = df['gender_cd'].map({'F': 0, 'M': 1}) # преобразование в бинарную переменную
df['emp_gender_cd'] = df['emp_gender_cd'].map({'W': 0, 'M': 1}) # преобразование в бинарную переменную
df['day_of_week'] = df['finish_dttm'].dt.dayofweek # создание переменной дня недели
df['holiday_flg'] = df.day_of_week.apply(lambda x: 1 if x > 4 else 0) # создание флага выходного дня
df['finish_dttm'] = df['finish_dttm'].dt.strftime('%Y-%m-%d') # сокращение структуры переменной времени
def million_city(region):
    if region == 'town' or region == 'rural':
        return 0
    if region == 'msk' or region == 'urban':
        return 1
    else:
        return np.nan
df['million_city'] = df['region_size'].apply(million_city) # создание флага города-миллионика

def child_flg(child):
    if child > 0:
        return 1
    else:
        return 0
df['child_flg'] = df['child_amt'].apply(child_flg) # создание флага наличия ребенка

credit_products = ['Credit Cards']
loan_products = ['Cash Loan', 'Secured Loan', 'Car Loan']
debit_investment_products = ['Debit Cards']
investment_products = ['Investment']
insurance_services = ['Casco', 'MVNO', 'Mortgage Insurance']

df['product'] = 'Другое'
df.loc[df['product_cd'].isin(credit_products), 'product'] = 'Кредитная карта'
df.loc[df['product_cd'].isin(loan_products), 'product'] = 'Кредит'
df.loc[df['product_cd'].isin(debit_investment_products), 'product'] = 'Дебетовая карта'
df.loc[df['product_cd'].isin(insurance_services), 'product'] = 'Страхование и услуги'
df.loc[df['product_cd'].isin(investment_products), 'product'] = 'Инвестиции'
df = df[~(df['product'] == 'Другое')]

loan_products = ['Upsell', 'Cold', 'Downsell', 'Utilization', 'Reutilization', 'Prolongation', 'Refinancing', 'Preapprove', 'Autoleads', 'BAF']
debit_investment_products = ['Afterfilling', 'Meeting', 'Incoming', 'Activation', 'Agreement']

df['function'] = 'Другое'
df.loc[df['functional_cd'].isin(loan_products), 'function'] = 'Продажи и предложения'

In [57]:
df.to_csv('data/data_preprocessed.csv', index=False)